<a href="https://colab.research.google.com/github/shashi-hue/Football_Data_manipulation/blob/main/Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from collections import defaultdict
import numpy as np

In [2]:
raw_df = pd.read_csv("/content/Football Data Test Task-DESKTOP-HKE84D6(Raw Data).csv")
raw_df['Date'] = pd.to_datetime(raw_df['Date'], dayfirst=False)
raw_df.sort_values('Date', inplace=True)
raw_df.reset_index(drop=True, inplace=True)

In [4]:
stats_to_compute = {
    "FTHG": "goals_scored",
    "FTAG": "goals_conceded",
    "HS": "shots",
    "AS": "opp_shots",
    "HST": "shots_on_target",
    "AST": "opp_shots_on_target",
    "HF": "fouls",
    "AF": "opp_fouls",
    "HC": "corners",
    "AC": "opp_corners",
    "HY": "yellow_cards",
    "AY": "opp_yellow_cards",
    "HR": "red_cards",
    "AR": "opp_red_cards"
}
result_types = ["W", "L", "D"]

rolling_windows = [5, 15, 38]

team_matches = defaultdict(list)

result_df = raw_df.copy()


In [5]:
for idx, row in raw_df.iterrows():
    home_team = row['HomeTeam']
    away_team = row['AwayTeam']
    ftr = row['FTR']

    features = {}

    for window in rolling_windows:

        past_matches_home = team_matches[home_team][-window:]
        features.update({
            f"FTHG_L{window}": sum(m['FTHG'] if m['home'] else m['FTAG'] for m in past_matches_home),
            f"HS_L{window}": sum(m['HS'] if m['home'] else m['AS'] for m in past_matches_home),
            f"HST_L{window}": sum(m['HST'] if m['home'] else m['AST'] for m in past_matches_home),
            f"HF_L{window}": sum(m['HF'] if m['home'] else m['AF'] for m in past_matches_home),
            f"HC_L{window}": sum(m['HC'] if m['home'] else m['AC'] for m in past_matches_home),
            f"HY_L{window}": sum(m['HY'] if m['home'] else m['AY'] for m in past_matches_home),
            f"HR_L{window}": sum(m['HR'] if m['home'] else m['AR'] for m in past_matches_home),
            f"HW_L{window}": sum(1 for m in past_matches_home if (m['FTR'] == 'H' and m['home']) or (m['FTR'] == 'A' and not m['home'])),
            f"HL_L{window}": sum(1 for m in past_matches_home if (m['FTR'] == 'A' and m['home']) or (m['FTR'] == 'H' and not m['home'])),
            f"HD_L{window}": sum(1 for m in past_matches_home if m['FTR'] == 'D')
        })


        past_matches_away = team_matches[away_team][-window:]
        features.update({
            f"FTAG_L{window}": sum(m['FTAG'] if m['home'] else m['FTHG'] for m in past_matches_away),
            f"AS_L{window}": sum(m['AS'] if m['home'] else m['HS'] for m in past_matches_away),
            f"AST_L{window}": sum(m['AST'] if m['home'] else m['HST'] for m in past_matches_away),
            f"AF_L{window}": sum(m['AF'] if m['home'] else m['HF'] for m in past_matches_away),
            f"AC_L{window}": sum(m['AC'] if m['home'] else m['HC'] for m in past_matches_away),
            f"AY_L{window}": sum(m['AY'] if m['home'] else m['HY'] for m in past_matches_away),
            f"AR_L{window}": sum(m['AR'] if m['home'] else m['HR'] for m in past_matches_away),
            f"AW_L{window}": sum(1 for m in past_matches_away if (m['FTR'] == 'A' and m['home']) or (m['FTR'] == 'H' and not m['home'])),
            f"AL_L{window}": sum(1 for m in past_matches_away if (m['FTR'] == 'H' and m['home']) or (m['FTR'] == 'A' and not m['home'])),
            f"AD_L{window}": sum(1 for m in past_matches_away if m['FTR'] == 'D')
        })


    for key, val in features.items():
        result_df.at[idx, key] = val


    team_matches[home_team].append({
        **row,
        "home": True
    })
    team_matches[away_team].append({
        **row,
        "home": False
    })


In [6]:
result_df.to_csv("Manipulated_Football_Data2.csv", index=False)
result_df.head()

,Incremental_ID,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,...,FTAG_L38,AS_L38,AST_L38,AF_L38,AC_L38,AY_L38,AR_L38,AW_L38,AL_L38,AD_L38
0,1,E0,2005-08-13,NaN,Aston Villa,Bolton,2,2,D,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,E0,2005-08-13,NaN,Everton,Man United,0,2,A,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,E0,2005-08-13,NaN,Fulham,Birmingham,0,0,D,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,E0,2005-08-13,NaN,Man City,West Brom,0,0,D,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,E0,2005-08-13,NaN,Middlesbrough,Liverpool,0,0,D,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
